In [1]:
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

#Change directory
os.chdir(r"C:\Users\Matome.Molokome\Desktop\Unisa\2021\HRCOS82 Research\favorita-grocery-sales-forecasting")

#Load Data
start_time = time.time()
chunk = pd.read_csv('train.csv',dtype={"id": object, "date": object,"store_nbr": object,"item_nbr": object,
                                         "unit_sales": float, "onpromotion": object
                                        },chunksize=1000000)


df = pd.concat(chunk)

end_time = time.time()

print ("Completed in",(end_time-start_time))


#df.sample(10)

Completed in 861.8004024028778


In [2]:
df.sample(10)
df.dtypes

KeyboardInterrupt: 

In [ ]:
stores = pd.read_csv('stores.csv',dtype={"store_nbr": object})

# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
stores ['type_numeric'] = labelencoder.fit_transform(stores ['type'])
stores ['city_numeric'] = labelencoder.fit_transform(stores ['city'])
stores ['state_numeric'] = labelencoder.fit_transform(stores ['state'])
stores

In [ ]:
oil = pd.read_csv('oil.csv')
print(oil)
oil.dtypes


In [ ]:
items = pd.read_csv('items.csv',dtype={"item_nbr": object})
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
items ['family_numeric'] = labelencoder.fit_transform(items ['family'])
print(items)
items.dtypes


In [ ]:
transactions = pd.read_csv('transactions.csv', dtype={"store_nbr": object})
print(transactions)
transactions.dtypes

In [ ]:
holidays_events = pd.read_csv('holidays_events.csv')
holidays_events ['Holiday'] = '1'
print(holidays_events)
holidays_events.dtypes

In [ ]:
#Filter Dataset
traindata = df[df.date >= '2017-08-01']
traindata.isnull().sum()

In [ ]:
#Encoding

#Dictionary
encoding_nums = {"onpromotion":     {"True": 1, "False": 0}}

#FindandReplace
traindata = traindata.replace(encoding_nums)

In [ ]:
traindata.sample(10)

In [ ]:
#Join Dimensions
traindata = pd.merge(traindata, items, on='item_nbr', how='left')
traindata = pd.merge(traindata, stores , on='store_nbr', how='left')
traindata = pd.merge(traindata, holidays_events , on='date', how='left')
traindata = pd.merge(traindata, oil , on='date', how='left')
traindata = pd.merge(traindata, transactions , on= ['date','store_nbr'], how='left')


traindata.sample(10)

In [ ]:
#drop columns
traindata.drop(['id', 'family','type_y','locale','locale_name','description','transferred','state','city','type_x'],
               axis=1,inplace=True)
traindata.sample(10)


In [ ]:
#Cleaning data 

#replace NAN with zero
traindata['Holiday'] = traindata['Holiday'].fillna(0)




In [ ]:
traindata.sample(10)

In [ ]:
#Cleaning data 
#replace NAN with mean
dcoilwtico_mean =traindata['dcoilwtico'].mean()
traindata['dcoilwtico'] = traindata['dcoilwtico'].fillna(dcoilwtico_mean)

In [ ]:
#Check negative sales
traindata[traindata['unit_sales'] < 0]

In [ ]:
#cleaning Data
# Drop Negative sales
traindata.drop(traindata[traindata['unit_sales'] < 0].index, inplace = True)

In [ ]:
#Check negative sales
traindata[traindata['unit_sales'] < 0]

In [ ]:
traindata.sample(10)

In [ ]:
#Feature Engineering
traindata['date'] = pd.to_datetime(traindata['date'], errors='coerce')
traindata ['month'] = traindata['date'].dt.month
traindata ['dayofthemonth'] = traindata['date'].dt.day
traindata ['dayofweek'] = traindata['date'].dt.dayofweek
traindata.sample(10)

In [ ]:
# Separate features and labels
X, y = traindata[['month','store_nbr', 'item_nbr','onpromotion','perishable','class','family_numeric','cluster','type_numeric','city_numeric','state_numeric','Holiday','dcoilwtico','transactions','dayofthemonth','dayofweek']].values, traindata['unit_sales'].values
print('Features:',X[:10], '\nLabels:', y[:10], sep='\n')

In [ ]:
from sklearn.model_selection import train_test_split

# Split data 70%-30% into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

print ('Training Set: %d rows\nTest Set: %d rows' % (X_train.shape[0], X_test.shape[0]))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, r2_score

# Use a Gradient Boosting algorithm
alg = GradientBoostingRegressor()

# Try these hyperparameter values
params = {
 'learning_rate': [0.1, 0.5, 1.0],
 'n_estimators' : [50, 100, 150]
 }

# Find the best hyperparameter combination to optimize the R2 metric
score = make_scorer(r2_score)
gridsearch = GridSearchCV(alg, params, scoring=score, cv=3, return_train_score=True)
gridsearch.fit(X_train, y_train)
print("Best parameter combination:", gridsearch.best_params_, "\n")

# Get the best model
model=gridsearch.best_estimator_
print(model, "\n")

# Evaluate the model using the test data
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
r2 = r2_score(y_test, predictions)
print("R2:", r2)

# Plot predicted vs actual
plt.scatter(y_test, predictions)
plt.xlabel('Actual Labels')
plt.ylabel('Predicted Labels')
plt.title('Resturant Sales Predictions')
# overlay the regression line
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()